In [1]:
from mainnet_launch.database.schema.full import *
from mainnet_launch.database.schema.postgres_operations import *
import plotly.express as px

from mainnet_launch.constants import *

from mainnet_launch.pages.rebalance_events.rebalance_events import _load_full_rebalance_event_df, make_expoded_box_plot

rebalance_df = _load_full_rebalance_event_df(AUTO_LRT)

2025-06-02 10:32:59.309 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-02 10:32:59.577 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:32:59.651 
  command:

    streamlit run /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-02 10:32:59.652 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:32:59.652 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:32:59.653 No runtime found, using MemoryCacheStorageManager
2025-06-02 10:33:00.157 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:33:00.158 Thread 'Thread-4

2025-06-02 10:33:00,362 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-02 10:33:00,363 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-02 10:33:00,508 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-02 10:33:00,509 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-02 10:33:00,656 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-02 10:33:00,656 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-02 10:33:00,816 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-02 10:33:00,817 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

2025-06-02 10:33:01.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:33:01.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:33:01.259 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:33:01.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:33:01.260 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-06-02 10:33:01,332 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-02 10:33:01,333 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-06-02 10:33:01,333 INFO sqlalchemy.engine.Engine [cached since 0.5167s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x126e3ea70>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-06-02 10:33:01,333 INFO sqlalchemy.engine.Engine SELECT
    destination_tokens.*,
    tokens.symbol,
    tokens.decimals
FROM destination_tokens
JOIN tokens
  ON destination_t

2025-06-02 10:33:01.572 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-02 10:33:01.573 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-06-02 10:33:01,660 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-02 10:33:01,661 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-06-02 10:33:01,661 INFO sqlalchemy.engine.Engine [cached since 0.8448s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x126d9fd30>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-06-02 10:33:01,661 INFO sqlalchemy.engine.Engine 
            SELECT *
            FROM destinations
            WHERE destinations.chain_id = 1
        
2025-06-02 10:33:01,6

In [2]:
rebalance_df.columns

Index(['tx_hash', 'autopool_vault_address', 'chain_id', 'rebalance_file_path',
       'destination_out', 'destination_in', 'quantity_out', 'quantity_in',
       'safe_value_out', 'safe_value_in', 'spot_value_in', 'spot_value_out',
       'swap_offset_period', 'block', 'datetime', 'destination_in_symbol',
       'destination_out_symbol', 'destination_in_tokens',
       'destination_out_tokens', 'tokens_move_name', 'spot_swap_cost',
       'spot_slippage_bps', 'safe_swap_cost', 'safe_slippage_bps', 'file_name',
       'datetime_generated', 'solver_address', 'rebalance_type', 'token_out',
       'token_in', 'move_name', 'amount_out', 'amount_out_safe_value',
       'min_amount_in', 'min_amount_in_safe_value', 'amount_out_spot_value',
       'out_dest_apr', 'min_amount_in_spot_value', 'in_dest_apr',
       'in_dest_adj_apr', 'apr_delta', 'num_candidate_destinations',
       'candidate_destinations_rank', 'projected_swap_cost',
       'projected_net_gain', 'projected_gross_gain', 'projected

In [4]:
rebalance_df["move_name"].value_counts()

move_name
autoLRT -> weeth-ng                   77
rsETH / WETH -> weeth-ng              44
autoLRT -> rsETH / WETH               25
autoLRT -> weETH/rETH                 17
weETH/rETH -> weeth-ng                12
autoLRT -> rsETH / ETHx               10
autoLRT -> weETH/ezETH/rswETH         10
weETH/rETH -> rsETH / WETH             9
weeth-ng -> rsETH / WETH               9
weeth-ng -> weETH/rETH                 4
weETH/ezETH/rswETH -> weeth-ng         3
rsETH / ETHx -> rsETH / WETH           2
weETH/rETH -> weETH/ezETH/rswETH       2
rsETH / WETH -> autoLRT                1
weETH/ezETH/rswETH -> rsETH / WETH     1
rsETH / ETHx -> autoLRT                1
weETH/ezETH/rswETH -> autoLRT          1
rsETH / WETH -> weETH/rETH             1
Name: count, dtype: int64

In [ ]:
# need date filtering
by_move_name = rebalance_df.groupby("move_name")[["safe_value_out", "safe_swap_cost"]].sum()
by_move_name["all_time_safe_slippage"] = 10_000 * by_move_name["safe_swap_cost"] / by_move_name["safe_value_out"]

px.bar(by_move_name["all_time_safe_slippage"])

px.scatter(by_move_name, x="safe_value_out", y="all_time_safe_slippage", color=by_move_name.index)

In [22]:
px.scatter(by_move_name, x="safe_value_out", y="all_time_safe_slippage", color=by_move_name.index)